In [ ]:
!pip install "audio-separator[gpu]"
!pip install -U openai-whisper
!pip install -U yt-dlp
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.40.3/install.sh | bash
!\. "$HOME/.nvm/nvm.sh"
!nvm install 24

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.2/385.2 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from audio_separator.separator import Separator
import whisper
model = whisper.load_model("turbo")
separator = Separator()
separator.load_model() #UVR-MDX-NET_Inst_82_beta.onnx"

100%|██████████████████████████████████████| 1.51G/1.51G [00:05<00:00, 278MiB/s]
INFO:audio_separator.separator.separator:Separator version 0.40.0 instantiating with output_dir: None, output_format: WAV
INFO:audio_separator.separator.separator:Output directory not specified. Using current working directory.
INFO:audio_separator.separator.separator:Using model directory from model_file_dir parameter: /tmp/audio-separator-models/
INFO:audio_separator.separator.separator:Operating System: Linux #1 SMP Thu Oct  2 10:42:05 UTC 2025
INFO:audio_separator.separator.separator:System: Linux Node: efc6f5053325 Release: 6.6.105+ Machine: x86_64 Proc: x86_64
INFO:audio_separator.separator.separator:Python Version: 3.12.12
INFO:audio_separator.separator.separator:PyTorch Version: 2.9.0+cu126
INFO:audio_separator.separator.separator:FFmpeg installed: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
INFO:audio_separator.separator.separator:ONNX Runtime GPU package in

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import textwrap

def make_text_image(top_text, middle_text, bottom_text, output_path="output.png", width=1280, height=720):
    # Make black background
    img = Image.new("RGB", (width, height), color="black")
    draw = ImageDraw.Draw(img)

    try:
        font_top_bottom = ImageFont.truetype("arial.ttf", size=40)
        font_middle = ImageFont.truetype("arial.ttf", size=60)
    except:
        font_top_bottom = ImageFont.load_default()
        font_middle = ImageFont.load_default()

    color_top_bottom = "#7F3F7F"
    color_middle = "#FF00FF"

    # function to measure text width/height
    def text_size(text, font):
        bbox = draw.textbbox((0, 0), text, font=font)
        return bbox[2] - bbox[0], bbox[3] - bbox[1]

    # function to make text 2 lines
    def wrap_text(text, font, max_width):
        return textwrap.wrap(text, width=int(len(text) * 0.67) or 1, break_long_words = False)

    middle_lines = wrap_text(middle_text, font_middle, width * 0.9)
    top_lines = wrap_text(top_text, font_middle, width * 0.9)
    bottom_lines = wrap_text(bottom_text, font_middle, width * 0.9)

    def total_height(lines, font, line_spacing=10):
        h_sum = sum(text_size(line, font)[1] for line in lines)
        h_sum += line_spacing * (len(lines) - 1)
        return h_sum

    top_height = total_height(top_lines, font_top_bottom)
    mid_height = total_height(middle_lines, font_middle)
    bot_height = total_height(bottom_lines, font_top_bottom)

    top_y = height * 0.1
    mid_y = (height - mid_height) / 2
    bot_y = height * 0.8

    def draw_centered(lines, font, color, y_start):
        y = y_start
        for line in lines:
            w, h = text_size(line, font)
            x = (width - w) / 2
            draw.text((x, y), line, font=font, fill=color)
            y += h + 10

    draw_centered(top_lines, font_top_bottom, color_top_bottom, top_y)
    draw_centered(middle_lines, font_middle, color_middle, mid_y)
    draw_centered(bottom_lines, font_top_bottom, color_top_bottom, bot_y)

    img.save(output_path)

def make_all_images(segments):
  size = len(segments)
  make_text_image('', '', segments[0]['text'], f"text_pre.png")
  make_text_image('', segments[0]['text'], segments[1]['text'], f"text_0.png")
  for i in range (1, size - 1):
      t_prev = segments[i - 1]['text']
      t = segments[i]['text']
      t_next = segments[i + 1]['text']
      make_text_image(t_prev, t, t_next, f"text_{i}.png")
  make_text_image(segments[size - 2]['text'], segments[size - 1]['text'], '', f"text_{size - 1}.png")

def make_durations_list(segments):
  with open("durations.txt", 'w') as f:
      f.write(f"file 'text_pre.png'\n")
      f.write(f"duration {segments[0]['start']}\n")
      for i in range(len(segments) - 2):
          duration = segments[i + 1]["start"] - segments[i]["start"]
          f.write(f"file 'text_{i}.png'\n")
          f.write(f"duration {duration}\n")
      f.write(f"file 'text_{len(segments) - 1}.png'\n")

In [ ]:
#!yt-dlp -x --audio-format wav -o "audio.%(ext)s" https://www.youtube.com/watch?v=dQw4w9WgXcQ
f = 'audio'
output_files = separator.separate(f + ".wav")
result = model.transcribe(f + ".wav") #"/content/" + f + "_(Vocals)_UVR-MDX-NET_Inst_82_beta.wav"
print(result["segments"])
segments = result["segments"]
make_all_images(segments)
make_durations_list(segments)
!ffmpeg -f concat -safe 0 -i durations.txt -i "/content/audio_(Instrumental)_model_bs_roformer_ep_317_sdr_12.wav" -map 0:v -map 1:a -vsync vfr -pix_fmt yuv420p -c:v libx264 output.mp4 -y

INFO:audio_separator.separator.separator:Processing file: dyad.wav
INFO:audio_separator.separator.separator:Starting separation process for audio_file_path: dyad.wav
INFO:audio_separator.separator.separator:Input audio subtype: PCM_16
INFO:audio_separator.separator.separator:Detected input bit depth: 16-bit
100%|██████████| 36/36 [01:57<00:00,  3.25s/it]
INFO:audio_separator.separator.separator:Saving Instrumental stem to dyad_(Instrumental)_model_bs_roformer_ep_317_sdr_12.wav...
INFO:audio_separator.separator.separator:Audio duration is 0.08 hours (282.32 seconds).
INFO:audio_separator.separator.separator:Using pydub for writing.
INFO:audio_separator.separator.separator:Writing output with 16-bit depth
INFO:audio_separator.separator.separator:Saving Vocals stem to dyad_(Vocals)_model_bs_roformer_ep_317_sdr_12.wav...
INFO:audio_separator.separator.separator:Audio duration is 0.08 hours (282.32 seconds).
INFO:audio_separator.separator.separator:Using pydub for writing.
INFO:audio_separa

[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 0.5, 'text': ' Huh!', 'tokens': [50365, 8063, 0, 50390], 'temperature': 0.0, 'avg_logprob': -0.2460463387625558, 'compression_ratio': 1.4530386740331491, 'no_speech_prob': 5.949269397433066e-13}, {'id': 1, 'seek': 0, 'start': 10.5, 'end': 12.5, 'text': " We've spent a lifetime apart", 'tokens': [50890, 492, 600, 4418, 257, 11364, 4936, 50990], 'temperature': 0.0, 'avg_logprob': -0.2460463387625558, 'compression_ratio': 1.4530386740331491, 'no_speech_prob': 5.949269397433066e-13}, {'id': 2, 'seek': 0, 'start': 12.5, 'end': 15.0, 'text': ' So I keep a record in my heart', 'tokens': [50990, 407, 286, 1066, 257, 2136, 294, 452, 1917, 51115], 'temperature': 0.0, 'avg_logprob': -0.2460463387625558, 'compression_ratio': 1.4530386740331491, 'no_speech_prob': 5.949269397433066e-13}, {'id': 3, 'seek': 0, 'start': 15.0, 'end': 17.5, 'text': " Of all the waiting and the wings I've done", 'tokens': [51115, 2720, 439, 264, 3806, 293, 264, 11405, 286, 600, 1

In [ ]:
import subprocess

def generate(link):
  subprocess.run(f'yt-dlp -x --audio-format wav -o "audio.%(ext)s" {link} --js-runtimes node', shell=True, check=True)
  f = 'audio'
  output_files = separator.separate(f + ".wav")
  result = model.transcribe(f + ".wav") #"/content/" + f + "_(Vocals)_UVR-MDX-NET_Inst_82_beta.wav"
  print(result["segments"])
  segments = result["segments"]
  make_all_images(segments)
  make_durations_list(segments)
  subprocess.run('ffmpeg -f concat -safe 0 -i durations.txt -i "/content/audio_(Instrumental)_model_bs_roformer_ep_317_sdr_12.wav" -map 0:v -map 1:a -vsync vfr -pix_fmt yuv420p -c:v libx264 output.mp4 -y', shell=True, check=True)
  return "output.mp4"


In [ ]:
generate('https://youtu.be/_AjJZEcMdww')

CalledProcessError: Command 'yt-dlp -x --audio-format wav -o "audio.%(ext)s" https://youtu.be/_AjJZEcMdww' returned non-zero exit status 1.

In [ ]:
import gradio as gr

demo = gr.Interface(
    fn=generate,
    inputs=["text"],
    outputs=["video"],
)

demo.launch()



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1cb12a26fe5f261daf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
